In [1]:
%cd ..

import cv2
import scipy
import numpy as np
from PIL import Image
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import matplotlib.pyplot as plt
from my_code import helpers
from my_code import models

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model, regularizers, backend
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import InputLayer, Lambda, LeakyReLU, ReLU, GlobalAveragePooling2D, BatchNormalization, Flatten, Dense, MaxPool2D, AveragePooling2D, Conv2D, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from utils import mnist_reader
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 28
NUM_CLASSES = 10
SEED = 2019
classes = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

/home/kuba/repos/fashion-mnist-assignment


# Calculate accuracy of the models on test & train dataset

In [3]:
def calculate_accuracy(X, y, model_name):
    X = X.reshape(X.shape[0], IMG_SIZE, IMG_SIZE, 1)
    y_encoded = to_categorical(y, num_classes=NUM_CLASSES, dtype='float32')

    generator_args = dict(
        data_format = 'channels_last',
        rescale=1./255,
    )

    datagen = ImageDataGenerator(**generator_args)
    datagen.fit(X)

    generator = datagen.flow(
        X, 
        y_encoded, 
        batch_size=256,
        shuffle=False
    )

    accuracies = []
    for i in range(5):
        model = tf.keras.models.load_model('models/model_{:}_{:}.h5'.format(model_name,i))
        result = model.evaluate_generator(generator)
        accuracies.append(result[1])
        print("Model {:} accuracy: {:.4f}".format(i,result[1]))
    
    print("\nMean accuracy {:.4f}".format(np.mean(accuracies)))


X, y = mnist_reader.load_mnist('data/fashion', kind='t10k')
calculate_accuracy(X,y,'v5')

X, y = mnist_reader.load_mnist('data/fashion', kind='train')
calculate_accuracy(X,y,'v5')

Model 0 accuracy: 0.9398
Model 1 accuracy: 0.9419
Model 2 accuracy: 0.9431
Model 3 accuracy: 0.9407
Model 4 accuracy: 0.9457

Mean accuracy 0.9422
Model 0 accuracy: 0.9718
Model 1 accuracy: 0.9651
Model 2 accuracy: 0.9690
Model 3 accuracy: 0.9676
Model 4 accuracy: 0.9749

Mean accuracy 0.9697


# Model ensemble

In [ ]:
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')
X_test = X_test.reshape(X_test.shape[0], IMG_SIZE, IMG_SIZE, 1)
y_test_encoded = to_categorical(y_test, num_classes=NUM_CLASSES, dtype='float32')

test_generator_args = dict(
    data_format = 'channels_last',
    rescale=1./255,
)

test_datagen = ImageDataGenerator(**test_generator_args)
test_datagen.fit(X_test)

batch_size = 200

test_generator = test_datagen.flow(
    X_test, 
    y_test_encoded, 
    batch_size=batch_size,
    shuffle=False
)

model_name = 'v5'
loaded_models = []
num_iters = len(test_generator)

# Load 5 trained models
for i in range(5):
    loaded_models.append(tf.keras.models.load_model('models/model_{:}_{:}.h5'.format(model_name,i)))
    print("Model {:} has been loaded".format(i))

preds  = np.empty((0,))
y_true = np.empty((0,))

# Iterate over the generator manually
for i,(x,y_batch) in enumerate(test_generator):
    print("Batch {:} out of {:}".format(i,num_iters))
    preds_batch = np.zeros((batch_size,NUM_CLASSES))
    for j in range(5):
        preds_batch += np.array(loaded_models[j].predict(x))
    
    preds  = np.hstack((preds, np.argmax(preds_batch,axis=-1)))
    y_true = np.hstack((y_true, np.argmax(y_batch,axis=-1)))
    
    # stop once iterated over all examples
    if (i+1) >= num_iters:
        break
        
acc = np.mean(preds==y_true)
print("Test accuracy {:.4f}\n".format(acc))

print(confusion_matrix(y_true, preds))